In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
dataset = ESOLDataset()
dataset.set_attributes()
dataset.map_list(method="set_edge_indices_reverse")

INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL:Read molecules from mol-file.
INFO:kgcnn.data.ESOL: ... process molecules 0 from 1128
INFO:kgcnn.data.ESOL: ... process molecules 1000 from 1128
INFO:kgcnn.mol.encoder:OneHotEncoder Symbol found ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
INFO:kgcnn.mol.encoder:OneHotEncoder Hybridization found [rdkit.Chem.rdchem.HybridizationType.SP3, rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.HybridizationType.SP2]
INFO:kgcnn.mol.encoder:OneHotEncoder TotalDegree found [2, 4, 1, 3]
INFO:kgcnn.mol.encoder:OneHotEncoder TotalNumHs found [1, 2, 0, 3, 4]
INFO:kgcnn.mol.encoder:OneHotEnc

In [3]:
from kgcnn.literature.DMPNN import make_model

In [4]:
model_config = {
    "name": "DMPNN",
    "inputs": [
        {"shape": [None, 41], "name": "node_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 11], "name": "edge_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 2], "name": "edge_indices", "dtype": "int64", "ragged": True},
        {"shape": [None, 1], "name": "edge_indices_reverse", "dtype": "int64", "ragged": True}
    ],
    "input_embedding": {"node": {"input_dim": 95, "output_dim": 64},
                        "edge": {"input_dim": 5, "output_dim": 64}},
    "pooling_args": {"pooling_method": "sum"},
    "edge_initialize": {"units": 128, "use_bias": True, "activation": "relu"},
    "edge_dense": {"units": 128, "use_bias": True, "activation": "linear"},
    "edge_activation": {"activation": "relu"},
    "node_dense": {"units": 128, "use_bias": True, "activation": "relu"},
    "verbose": 10, "depth": 5,
    "dropout": {"rate": 0.1},
    "output_embedding": "graph",
    "output_mlp": {
        "use_bias": [True, True, False], "units": [64, 32, 1],
        "activation": ["relu", "relu", "linear"]
    }
}
# Test making model
model = make_model(**model_config)

INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}, 'graph': {'input_dim': 100, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activation': 'linear'}, 'edge_activation': {'activation': 'relu'}, 'node_dense': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'verbose': 10, 'depth': 5, 'dropout': {'rate': 0.1}, 'output_embedding': '

In [5]:
dataset.clean(model_config["inputs"])

INFO:kgcnn.data.ESOL:Property edge_attributes is an empty list for graph 934.
INFO:kgcnn.data.ESOL:Property edge_indices is an empty list for graph 934.
INFO:kgcnn.data.ESOL:Property edge_indices_reverse is an empty list for graph 934.


array([934])

In [6]:
labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
labels.shape

(1127, 1)

In [7]:
from copy import deepcopy
from sklearn.model_selection import train_test_split
from keras.backend import clear_session

In [8]:
from kgcnn.training.optimizer import Adan
from tensorflow.keras.optimizers import Adam, SGD, Adamax, Adadelta

In [9]:
train_index, test_index = train_test_split(np.arange(len(dataset)), test_size=0.25, random_state=42)
x_train, y_train = dataset[train_index].tensor(model_config["inputs"]), labels[train_index]
x_valid, y_valid = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]

In [ ]:
hist_opt = []
# Adan
opt_selection = [
    Adan(), Adam(), 
    SGD(), Adamax(), Adadelta()]

for opt in opt_selection:
    model = make_model(**model_config)
    print(opt)
    model.compile(
        loss="mean_absolute_error",
        optimizer=opt,
        metrics=["mean_absolute_error"],
    )
    hist_run = model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=32,
        epochs=300,
    )
    hist_opt.append(hist_run)

INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}, 'graph': {'input_dim': 100, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activation': 'linear'}, 'edge_activation': {'activation': 'relu'}, 'node_dense': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'verbose': 10, 'depth': 5, 'dropout': {'rate': 0.1}, 'output_embedding': '

Epoch 1/300


C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_11/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_11/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_11/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_9/gather_nodes_outgoing_11/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_9/gather_nodes_outgoing_11/Reshape:0", shape=(Non

27/27 [==============================] - 7s 76ms/step - loss: 2.0856 - mean_absolute_error: 2.0856 - val_loss: 1.1591 - val_mean_absolute_error: 1.1591
Epoch 2/300
27/27 [==============================] - 1s 43ms/step - loss: 1.0265 - mean_absolute_error: 1.0265 - val_loss: 0.8844 - val_mean_absolute_error: 0.8844
Epoch 3/300
27/27 [==============================] - 1s 41ms/step - loss: 0.8624 - mean_absolute_error: 0.8624 - val_loss: 0.7943 - val_mean_absolute_error: 0.7943
Epoch 4/300
27/27 [==============================] - 1s 42ms/step - loss: 0.8211 - mean_absolute_error: 0.8211 - val_loss: 0.7779 - val_mean_absolute_error: 0.7779
Epoch 5/300
27/27 [==============================] - 1s 40ms/step - loss: 0.7631 - mean_absolute_error: 0.7631 - val_loss: 0.7466 - val_mean_absolute_error: 0.7466
Epoch 6/300
27/27 [==============================] - 1s 43ms/step - loss: 0.7411 - mean_absolute_error: 0.7411 - val_loss: 0.7292 - val_mean_absolute_error: 0.7292
Epoch 7/300
27/27 [=========

Epoch 51/300
27/27 [==============================] - 1s 32ms/step - loss: 0.3324 - mean_absolute_error: 0.3324 - val_loss: 0.5011 - val_mean_absolute_error: 0.5011
Epoch 52/300
27/27 [==============================] - 1s 32ms/step - loss: 0.3217 - mean_absolute_error: 0.3217 - val_loss: 0.4789 - val_mean_absolute_error: 0.4789
Epoch 53/300
27/27 [==============================] - 1s 32ms/step - loss: 0.3260 - mean_absolute_error: 0.3260 - val_loss: 0.5071 - val_mean_absolute_error: 0.5071
Epoch 54/300
27/27 [==============================] - 1s 33ms/step - loss: 0.3295 - mean_absolute_error: 0.3295 - val_loss: 0.4758 - val_mean_absolute_error: 0.4758
Epoch 55/300
27/27 [==============================] - 1s 34ms/step - loss: 0.3210 - mean_absolute_error: 0.3210 - val_loss: 0.4886 - val_mean_absolute_error: 0.4886
Epoch 56/300
27/27 [==============================] - 1s 34ms/step - loss: 0.3141 - mean_absolute_error: 0.3141 - val_loss: 0.4738 - val_mean_absolute_error: 0.4738
Epoch 57/3

...

27/27 [==============================] - 1s 34ms/step - loss: 0.1307 - mean_absolute_error: 0.1307 - val_loss: 0.4881 - val_mean_absolute_error: 0.4881
Epoch 298/300
27/27 [==============================] - 1s 38ms/step - loss: 0.1349 - mean_absolute_error: 0.1349 - val_loss: 0.4838 - val_mean_absolute_error: 0.4838
Epoch 299/300
27/27 [==============================] - 1s 37ms/step - loss: 0.1339 - mean_absolute_error: 0.1339 - val_loss: 0.4696 - val_mean_absolute_error: 0.4696
Epoch 300/300
27/27 [==============================] - 1s 35ms/step - loss: 0.1357 - mean_absolute_error: 0.1357 - val_loss: 0.4844 - val_mean_absolute_error: 0.4844


INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}, 'graph': {'input_dim': 100, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activation': 'linear'}, 'edge_activation': {'activation': 'relu'}, 'node_dense': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'verbose': 10, 'depth': 5, 'dropout': {'rate': 0.1}, 'output_embedding': '

Epoch 1/300


C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_17/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_17/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_17/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_14/gather_nodes_outgoing_17/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_14/gather_nodes_outgoing_17/Reshape:0", shape=(N

27/27 [==============================] - 5s 62ms/step - loss: 2.2410 - mean_absolute_error: 2.2410 - val_loss: 1.0319 - val_mean_absolute_error: 1.0319
Epoch 2/300
27/27 [==============================] - 1s 27ms/step - loss: 1.1753 - mean_absolute_error: 1.1753 - val_loss: 0.9732 - val_mean_absolute_error: 0.9732
Epoch 3/300
27/27 [==============================] - 1s 27ms/step - loss: 1.0262 - mean_absolute_error: 1.0262 - val_loss: 0.8795 - val_mean_absolute_error: 0.8795
Epoch 4/300
27/27 [==============================] - 1s 29ms/step - loss: 1.0423 - mean_absolute_error: 1.0423 - val_loss: 1.1544 - val_mean_absolute_error: 1.1544
Epoch 5/300
27/27 [==============================] - 1s 27ms/step - loss: 0.9130 - mean_absolute_error: 0.9130 - val_loss: 0.8360 - val_mean_absolute_error: 0.8360
Epoch 6/300
27/27 [==============================] - 1s 28ms/step - loss: 0.7925 - mean_absolute_error: 0.7925 - val_loss: 0.7038 - val_mean_absolute_error: 0.7038
Epoch 7/300
27/27 [=========

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=(10,5))
for hist_run, opt in zip(hist_opt, opt_selection):
    axs[0].plot(hist_run.history["loss"], label=opt.get_config()["name"])
    axs[0].set_ytitle("Loss")
    axs[0].set_xtitle("Epochs")
    axs[0].legend()
for hist_run, opt in zip(hist_opt, opt_selection):
    axs[1].plot(hist_run.history["val_loss"], label=opt.get_config()["name"])
    axs[1].set_ytitle("Validation Loss")
    axs[1].set_xtitle("Epochs")
    axs[1].legend()